In [1]:
# enable reload the module
%load_ext autoreload
%autoreload 2

In [20]:
# Import the necessary modules
import argparse
import datetime
import glob
import os
import re
import time
from pathlib import Path

import numpy as np
import torch
from tensorboardX import SummaryWriter

from eval_utils import eval_utils
from mtr.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from mtr.datasets import build_dataloader
from mtr.models import model as model_utils
from mtr.utils import common_utils
from mtr.datasets.waymo.waymo_dataset import WaymoDataset
from torch.utils.data import DataLoader


In [26]:
cfg_file = '/hdd/Git_Repo/MTR/tools/cfgs/waymo/mtr_eval.yaml'

cfg_from_yaml_file(cfg_file, cfg)
cfg.TAG = Path(cfg_file).stem
cfg.EXP_GROUP_PATH = '/'.join(cfg_file.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'

np.random.seed(1024)

# Set the output directory
output_dir = cfg.ROOT_DIR / 'output' / 'val_interactive'
output_dir.mkdir(parents=True, exist_ok=True)

eval_output_dir = output_dir / 'eval'
eval_output_dir.mkdir(parents=True, exist_ok=True)

log_file = eval_output_dir / ('log_eval_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)


In [32]:
# Build dataset
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    batch_size=4,
    dist=False, workers=1, logger=logger, training=False
)

2023-05-15 21:21:47,528   INFO  Start to load infos from /home/zixu/Data/Dataset/MTR_dataset/processed_scenarios_val_interactive_infos.pkl
2023-05-15 21:21:47,539   INFO  Total scenes before filters: 1441
2023-05-15 21:21:47,561   INFO  Total scenes after filter_info_by_object_type: 1441
2023-05-15 21:21:47,563   INFO  Total scenes after filters: 1441


/home/zixu/Data/Dataset/MTR_dataset/processed_scenarios_val_interactive_infos.pkl


In [31]:
# load model
model = model_utils.MotionTransformer(config=cfg.MODEL)
ckpt_path = '/hdd/Git_Repo/MTR/model/checkpoint_epoch_30.pth'
it, epoch = model.load_params_from_file(filename=ckpt_path, logger=logger, to_cpu=False)
model = model.cuda()

2023-05-15 21:21:21,714   INFO  ==> Loading parameters from checkpoint /hdd/Git_Repo/MTR/model/checkpoint_epoch_30.pth to GPU
2023-05-15 21:21:22,111   INFO  ==> Checkpoint trained from version: none
2023-05-15 21:21:22,112   INFO  The number of disk ckpt keys: 842
2023-05-15 21:21:22,160   INFO  Missing keys: []
2023-05-15 21:21:22,160   INFO  The number of missing keys: 0
2023-05-15 21:21:22,161   INFO  The number of unexpected keys: 0
2023-05-15 21:21:22,161   INFO  ==> Done (total keys 842)


In [33]:
eval_utils.eval_one_epoch(
    cfg, model, test_loader, 30, logger, dist_test=False,
    result_dir=eval_output_dir, save_to_file=True
)

2023-05-15 21:22:12,543   INFO  *************** EPOCH 30 EVALUATION *****************
eval:   0%|          | 0/361 [00:00<?, ?it/s]2023-05-15 21:22:13,389   INFO  eval: epoch=30, batch_iter=0/361, batch_size=4, iter_cost=0.84s, time_cost: 00:00/05:03, 
2023-05-15 21:22:19,941   INFO  eval: epoch=30, batch_iter=50/361, batch_size=4, iter_cost=0.15s, time_cost: 00:07/00:45, 
2023-05-15 21:22:26,392   INFO  eval: epoch=30, batch_iter=100/361, batch_size=4, iter_cost=0.14s, time_cost: 00:13/00:36, 
2023-05-15 21:22:32,884   INFO  eval: epoch=30, batch_iter=150/361, batch_size=4, iter_cost=0.14s, time_cost: 00:20/00:28, 
2023-05-15 21:22:39,282   INFO  eval: epoch=30, batch_iter=200/361, batch_size=4, iter_cost=0.13s, time_cost: 00:26/00:21, 
2023-05-15 21:22:45,746   INFO  eval: epoch=30, batch_iter=250/361, batch_size=4, iter_cost=0.13s, time_cost: 00:33/00:14, 
2023-05-15 21:22:51,865   INFO  eval: epoch=30, batch_iter=300/361, batch_size=4, iter_cost=0.13s, time_cost: 00:39/00:07, 
2023

Total number for evaluation (intput): 2882
Total number for evaluation (after processed): 2882


2023-05-15 21:23:01,754   INFO  
minADE - TYPE_VEHICLE_5	: 0.2790 
minADE - TYPE_VEHICLE_9	: 0.5900 
minADE - TYPE_VEHICLE_15	: 1.2319 
minADE - TYPE_PEDESTRIAN_5	: 0.1877 
minADE - TYPE_PEDESTRIAN_9	: 0.3656 
minADE - TYPE_PEDESTRIAN_15	: 0.7022 
minADE - TYPE_CYCLIST_5	: 0.3820 
minADE - TYPE_CYCLIST_9	: 0.6706 
minADE - TYPE_CYCLIST_15	: 1.1185 
minFDE - TYPE_VEHICLE_5	: 0.5006 
minFDE - TYPE_VEHICLE_9	: 1.1610 
minFDE - TYPE_VEHICLE_15	: 2.5731 
minFDE - TYPE_PEDESTRIAN_5	: 0.3550 
minFDE - TYPE_PEDESTRIAN_9	: 0.7514 
minFDE - TYPE_PEDESTRIAN_15	: 1.4656 
minFDE - TYPE_CYCLIST_5	: 0.6952 
minFDE - TYPE_CYCLIST_9	: 1.1891 
minFDE - TYPE_CYCLIST_15	: 2.0326 
MissRate - TYPE_VEHICLE_5	: 0.0754 
MissRate - TYPE_VEHICLE_9	: 0.1231 
MissRate - TYPE_VEHICLE_15	: 0.1857 
MissRate - TYPE_PEDESTRIAN_5	: 0.0561 
MissRate - TYPE_PEDESTRIAN_9	: 0.1000 
MissRate - TYPE_PEDESTRIAN_15	: 0.1150 
MissRate - TYPE_CYCLIST_5	: 0.2453 
MissRate - TYPE_CYCLIST_9	: 0.2308 
MissRate - TYPE_CYCLIST_15	: 0.1

{'minADE - TYPE_VEHICLE_5\t': 0.27896222472190857,
 'minADE - TYPE_VEHICLE_9\t': 0.5899892449378967,
 'minADE - TYPE_VEHICLE_15\t': 1.2319161891937256,
 'minADE - TYPE_PEDESTRIAN_5\t': 0.18770670890808105,
 'minADE - TYPE_PEDESTRIAN_9\t': 0.36557021737098694,
 'minADE - TYPE_PEDESTRIAN_15\t': 0.7021687626838684,
 'minADE - TYPE_CYCLIST_5\t': 0.38200634717941284,
 'minADE - TYPE_CYCLIST_9\t': 0.6706194281578064,
 'minADE - TYPE_CYCLIST_15\t': 1.1184755563735962,
 'minFDE - TYPE_VEHICLE_5\t': 0.5005630850791931,
 'minFDE - TYPE_VEHICLE_9\t': 1.1610372066497803,
 'minFDE - TYPE_VEHICLE_15\t': 2.573063373565674,
 'minFDE - TYPE_PEDESTRIAN_5\t': 0.35500088334083557,
 'minFDE - TYPE_PEDESTRIAN_9\t': 0.7513847351074219,
 'minFDE - TYPE_PEDESTRIAN_15\t': 1.4656437635421753,
 'minFDE - TYPE_CYCLIST_5\t': 0.6951568126678467,
 'minFDE - TYPE_CYCLIST_9\t': 1.1891125440597534,
 'minFDE - TYPE_CYCLIST_15\t': 2.032588481903076,
 'MissRate - TYPE_VEHICLE_5\t': 0.07544957101345062,
 'MissRate - TYPE_VE